## BC205 Exercise 4: Burrows-Wheeler Transformation
---

#####  **Author:** Georgios Kousis Tsampazis
---

### Overview
This exercise is split in two parts and will be performed with a partner. You will not work in pairs but you need a pair. Each pair A,B will work in the following way:
1. Member A will browse Uniprot (https://www.uniprot.org/) for a protein sequence of his/her choice. Then:
  a. He/She will obtain the Protein Accession Number and the fasta sequence of the protein 
  b. Will use the BWT to transform it using the dollar (\$) sign for terminal symbol. \$ will have top priority in sorting.
  c. He/She will send the **BWT Transform of the sequence** to member B.
2. Member B will have to: a. Implement the BWT reversal (decoding) algorithm to find the actual sequence A sent him/her b. BLAST it against uniprot (with Protein BLAST) and identify the accession number
   
Once this is complete (or the deadline has arrived) A will send me the Accession Number and Fasta sequence of the protein and B will send me the Accession Number he/she identified. Both will also send me the code. Please do so independently even if you have cross-compared your results.

### Member A 


### Noteboook dependencies
*As always a conda enviroment is recommended for easier installation*

In [1]:
pip install tqdm Bio pandas

Note: you may need to restart the kernel to use updated packages.


### Downlaod data

In [2]:
import os
import requests
from tqdm import tqdm

# Create the data directory if it doesn't exist.
data_dir = "./data_tmp"
os.makedirs(data_dir, exist_ok=True)
files_to_download = {"P05814.fa": "https://rest.uniprot.org/uniprotkb/P05814.fasta"}

def download_file(url, file_path):
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raise an error for bad status codes.
    
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kilobyte blocks

    with open(file_path, 'wb') as file, tqdm(
            total=total_size, unit='B', unit_scale=True, desc=os.path.basename(file_path)
        ) as progress_bar:
        for data in response.iter_content(block_size):
            file.write(data)
            progress_bar.update(len(data))

for filename, url in files_to_download.items():
    file_path = os.path.join(data_dir, filename)
    if not os.path.exists(file_path):
        print(f"Downloading {filename} ...")
        try:
            download_file(url, file_path)
            print(f"{filename} downloaded successfully.")
        except Exception as e:
            print(f"Error downloading {filename}: {e}")
    else:
        print(f"{filename} already exists. Skipping download.")
print("Files are ready.")


P05814.fa already exists. Skipping download.
Files are ready.


### Open the sequences & add \$
*I find it funny using a library just to open a fasta*

In [3]:
from Bio import SeqIO
record = SeqIO.read("./data_tmp/P05814.fa", "fasta")
protein = record.seq
protein_with_terminator = protein + "$" # If you don't do that u will get punished later

### Protein Sequence

In [14]:
print(protein_with_terminator)

MKVLILACLVALALARETIESLSSSEESITEYKQKVEKVKHEDQQQGEDEHQDKIYPSFQPQPLIYPFVEPIPYGFLPQNILPLAQPAVVLPVPQPEIMEVPKAKDTVYTKGRVMPVLKSPTIPFFDPQIPKLTDLENLHLPLPLLQPLMQQVPQPIPQTLALPPQPLWSVPQPKVLPIPQQVVPYPQRAVPVQALLLNQELLLNPTHQIYPVTQPLAPVHNPISV$


###  Burrows–Wheeler Transform Function
**Function:** `bwt_transform(s)`

**Purpose:**
Compute the Burrows–Wheeler Transform (BWT) of an input string by sorting all of its cyclic rotations and extracting the last column of the sorted matrix. \
**Parameters:**
* `s` (`str`)
  An input string (in our case protein sequence) that **must** include a unique terminal symbol (commonly `$`) at its end. The terminal symbol ensures all rotations are distinct and that the transform is reversible. 

**Returns:**
* (`str`)
  The BWT of the input

**Workflow:**

1. **Generate all cyclic rotations**
2. **Sort the rotations lexicographically**
3. **Extract the last column**
4. **Return the BWT string**

**Usage Example:**

```python
seq_ex = "GATTACA$"
bwt = bwt_transform(seq_ex)
```


In [9]:
# Very useful 
class YouSuckError(Exception):
    """A very informative error raise"""
    pass



def bwt_transform(s):
    """
    Computes the BWT of the string s (which must include a unique '$' at the end).
    """
    #### Unexpected Input
    if s.count("$")!=1:
         raise YouSuckError(
         f"You had 1 Job and you failed!\n" 
         f"Your string either has no $ or has more than one.\n"
         f"I am computer program and in reality I could tell you how many $ (if any) there are...\n"
         f"...but I wont.")
    ####
    global rotations
    rotations = [s[i:] + s[:i] for i in range(len(s))]
    rotations.sort()  
    return ''.join(rotation[-1] for rotation in rotations)

#### Execution: call the `bwt_transform` and print results

In [11]:
bwt_transform(protein_with_terminator)

'VLKVLQLLLLRPAEQTFEKGHSDPVQLVEIRMTFPGSPQYKKLVETTLNETQPPPPSLKQPATVDPYLQEMPIATPPADNVPVEALLPPLLVIPHAFVLSKCPI$VLQEHLLQQIYLQENVIQQLQLQLQLDLVVVQPIYIYNSAMLVYIVVHNQPHKPAPPPTPLPFQDMPPPQQQAGSPEEKSLIWLIPEPYQVDSLKFPKKPKVREPQSAVPPAQTLPEIPIIV'

#### Detailed Rotations

In [12]:
import pandas as pd
df = pd.DataFrame({
    "Rotation": rotations,
    "Last Character": [r[-1] for r in rotations]
})
print(df)

                                              Rotation Last Character
0    ($, M, K, V, L, I, L, A, C, L, V, A, L, A, L, ...              V
1    (A, C, L, V, A, L, A, L, A, R, E, T, I, E, S, ...              L
2    (A, K, D, T, V, Y, T, K, G, R, V, M, P, V, L, ...              K
3    (A, L, A, L, A, R, E, T, I, E, S, L, S, S, S, ...              V
4    (A, L, A, R, E, T, I, E, S, L, S, S, S, E, E, ...              L
..                                                 ...            ...
222  (Y, P, F, V, E, P, I, P, Y, G, F, L, P, Q, N, ...              I
223  (Y, P, Q, R, A, V, P, V, Q, A, L, L, L, N, Q, ...              P
224  (Y, P, S, F, Q, P, Q, P, L, I, Y, P, F, V, E, ...              I
225  (Y, P, V, T, Q, P, L, A, P, V, H, N, P, I, S, ...              I
226  (Y, T, K, G, R, V, M, P, V, L, K, S, P, T, I, ...              V

[227 rows x 2 columns]


#### Funny Execution: call the `bwt_transform` and print **funny** results

In [13]:
bwt_transform(protein)

YouSuckError: You had 1 Job and you failed!
Your string either has no $ or has more than one.
I am computer program and in reality I could tell you how many $ (if any) there are...
...but I wont.

---
### For Member B
#### To decipher:
https://pastebin.com/4x8NmKDU


---  
*Here lies the answear*  
*shhhhh*

> **UniProt accession**: [P05814](https://rest.uniprot.org/uniprotkb/P05814.fasta)
